In [1]:
#iris test
import sklearn
from sklearn import datasets
import matplotlib.pyplot as plt
# 嗯...這環境沒 tensorflow 也沒 keras ，反正弄 iris 而已，直接用 sklearn 跑吧
#import tensorflow
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers.core import Dense, Activation
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

from sklearn.neural_network import MLPClassifier

import mlflow
import mlflow.sklearn

In [2]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
print(X[0:5])
print(y[0:5])
#X做scale
X = preprocessing.scale(X)
print(X[0:5])
#y轉one hot
oneHotEncoder = preprocessing.OneHotEncoder()
y = oneHotEncoder.fit_transform(y.reshape(-1, 1)).toarray() #OneHotEncoder吃2D，然後y是1D，所以要reshape
print(y[0:5])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
[0 0 0 0 0]
[[-0.90068117  1.01900435 -1.34022653 -1.3154443 ]
 [-1.14301691 -0.13197948 -1.34022653 -1.3154443 ]
 [-1.38535265  0.32841405 -1.39706395 -1.3154443 ]
 [-1.50652052  0.09821729 -1.2833891  -1.3154443 ]
 [-1.02184904  1.24920112 -1.34022653 -1.3154443 ]]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [3]:
#設置 mlflow experiment
try:
    experiment_id = mlflow.create_experiment(name = "test_iris")
except:
    print("Experiment test_iris already exists")
    experiment_id = '1'
experiment = mlflow.get_experiment(experiment_id)
print("Name: {}".format(experiment.name))
print("Experiment_id: {}".format(experiment.experiment_id))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Tags: {}".format(experiment.tags))
print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))

Experiment test_iris already exists
Name: test_iris
Experiment_id: 1
Artifact Location: file:///home/jovyan/work/mlruns/1
Tags: {}
Lifecycle_stage: active


In [4]:
#訓練
layer_sizes = [(10, 10, 10), (10, 20), (20, 40)]
solvers = ["adam", "lbfgs"]
mlps = []


for sizeArg in layer_sizes:
    for solverArg in solvers:
        with mlflow.start_run(experiment_id = experiment_id):
            mlp = MLPClassifier(max_iter = 1000, solver = solverArg, alpha = 0.001, hidden_layer_sizes = sizeArg, random_state = 1234)
            mlp.fit(X_train, y_train)
            mlflow.log_param("sizeArg", sizeArg)
            mlflow.log_param("solverArg", solverArg)
            mlflow.log_metric("loss", mlp.loss_)
            mlflow.log_metric("score", mlp.score(X_train, y_train))
            print(mlp)
            print("loss = %f" % (mlp.loss_))
            mlflow.sklearn.log_model(mlp, "model")
            mlps.append(mlp)

MLPClassifier(alpha=0.001, hidden_layer_sizes=(10, 10, 10), max_iter=1000,
              random_state=1234)
loss = 0.095996
MLPClassifier(alpha=0.001, hidden_layer_sizes=(10, 10, 10), max_iter=1000,
              random_state=1234, solver='lbfgs')
loss = 0.056494
MLPClassifier(alpha=0.001, hidden_layer_sizes=(10, 20), max_iter=1000,
              random_state=1234)
loss = 0.100154
MLPClassifier(alpha=0.001, hidden_layer_sizes=(10, 20), max_iter=1000,
              random_state=1234, solver='lbfgs')
loss = 0.050242
MLPClassifier(alpha=0.001, hidden_layer_sizes=(20, 40), max_iter=1000,
              random_state=1234)
loss = 0.084833
MLPClassifier(alpha=0.001, hidden_layer_sizes=(20, 40), max_iter=1000,
              random_state=1234, solver='lbfgs')
loss = 0.001007


In [5]:
#預測
for mlp in mlps:
    predict_train = mlp.predict(X_train)
    predict_test = mlp.predict(X_test)
    train_acc = sklearn.metrics.accuracy_score(predict_train, y_train)
    test_acc = sklearn.metrics.accuracy_score(predict_test, y_test)
    print(mlp)
    print("train accuracy: %f" % (train_acc))
    print("test accuracy: %f\n\n" % (test_acc))

MLPClassifier(alpha=0.001, hidden_layer_sizes=(10, 10, 10), max_iter=1000,
              random_state=1234)
train accuracy: 0.975000
test accuracy: 0.966667


MLPClassifier(alpha=0.001, hidden_layer_sizes=(10, 10, 10), max_iter=1000,
              random_state=1234, solver='lbfgs')
train accuracy: 0.983333
test accuracy: 0.966667


MLPClassifier(alpha=0.001, hidden_layer_sizes=(10, 20), max_iter=1000,
              random_state=1234)
train accuracy: 0.975000
test accuracy: 0.966667


MLPClassifier(alpha=0.001, hidden_layer_sizes=(10, 20), max_iter=1000,
              random_state=1234, solver='lbfgs')
train accuracy: 0.983333
test accuracy: 0.966667


MLPClassifier(alpha=0.001, hidden_layer_sizes=(20, 40), max_iter=1000,
              random_state=1234)
train accuracy: 0.991667
test accuracy: 0.966667


MLPClassifier(alpha=0.001, hidden_layer_sizes=(20, 40), max_iter=1000,
              random_state=1234, solver='lbfgs')
train accuracy: 1.000000
test accuracy: 0.966667


